In [2]:
# 🚗 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 📦 Instalar librerías necesarias
!pip install lightgbm optuna imbalanced-learn pyarrow --quiet

# 📚 Importar librerías
import pandas as pd
import numpy as np
import optuna
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tqdm import tqdm

# 📂 Cargar base completa
ruta = "/content/drive/MyDrive/Datos/6_Base_Modelos_Predictivos.parquet"
df = pd.read_parquet(ruta)

# ➕ Calcular número de años por empresa
df["n_anios"] = df.groupby("NIT")["Año"].transform("count")

# 🧼 Limpieza de columnas categóricas
X = df.drop(columns=["RQ", "NIT"])
y = df["RQ"]
X = pd.get_dummies(X, columns=["DEP", "CIIU_Letra"], drop_first=True)
X = X.replace([np.inf, -np.inf], 1e6)

# ⚙️ Definir función objetivo para Optuna
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "random_state": 42
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = []

    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        sm = SMOTE(random_state=42)
        X_res, y_res = sm.fit_resample(X_train, y_train)

        model = LGBMClassifier(**params)
        model.fit(X_res, y_res)
        y_pred = model.predict(X_test)
        f1_scores.append(f1_score(y_test, y_pred))

    return np.mean(f1_scores)

# 🚀 Ejecutar búsqueda con Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

# 🏆 Mostrar mejores parámetros
print("\n✅ Mejores parámetros encontrados:")
print(study.best_params)
print(f"🔝 Mejor F1-score: {study.best_value:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[I 2025-05-19 02:33:06,046] A new study created in memory with name: no-name-4f58a73d-300d-4bfb-99e2-bdf34f8cdf38


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 36448, number of negative: 36448
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024509 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11630
[LightGBM] [Info] Number of data points in the train set: 72896, number of used features: 93
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 36449, number of negative: 36449
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032047 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11630
[Lig

[I 2025-05-19 02:33:37,631] Trial 0 finished with value: 0.4740733412333613 and parameters: {'n_estimators': 331, 'max_depth': 7, 'learning_rate': 0.05655616343150092, 'subsample': 0.7079493567598145, 'colsample_bytree': 0.9396184882665024}. Best is trial 0 with value: 0.4740733412333613.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 36448, number of negative: 36448
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.056458 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11630
[LightGBM] [Info] Number of data points in the train set: 72896, number of used features: 93
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 36449, number of negative: 36449
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.060747 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11630
[LightGBM] [Info] Number of data points in the train set: 72898, number of used features: 93
[LightGBM] [Info] [binary:BoostFromScor

...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 36448, number of negative: 36448
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.055843 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11630
[LightGBM] [Info] Number of data points in the train set: 72896, number of used features: 93
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 36449, number of negative: 36449
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058112 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11630
[LightGBM] [Info] Number of data points in the train set: 72898, number of used features: 93
[LightGBM] [Info] [binary:BoostFromScor

[I 2025-05-19 02:47:56,087] Trial 28 finished with value: 0.5085738436705138 and parameters: {'n_estimators': 358, 'max_depth': 7, 'learning_rate': 0.22784743627633405, 'subsample': 0.7431003020034357, 'colsample_bytree': 0.8208783937553302}. Best is trial 1 with value: 0.5161875088994918.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 36448, number of negative: 36448
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023221 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11630
[LightGBM] [Info] Number of data points in the train set: 72896, number of used features: 93
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive

[I 2025-05-19 02:48:21,938] Trial 29 finished with value: 0.5023087749495527 and parameters: {'n_estimators': 314, 'max_depth': 6, 'learning_rate': 0.27019172487657894, 'subsample': 0.740365709142714, 'colsample_bytree': 0.931066717090171}. Best is trial 1 with value: 0.5161875088994918.



✅ Mejores parámetros encontrados:
{'n_estimators': 429, 'max_depth': 9, 'learning_rate': 0.19241269896238575, 'subsample': 0.6518239170936834, 'colsample_bytree': 0.8789471428121596}
🔝 Mejor F1-score: 0.5162
